In [309]:
import pandas as pd
from pathlib import Path
import os
import datetime

In [442]:
sheet_id = "1vQJ3psvi0Kl6_tecq3SV_mCMuQPvnXxnQ2MLN2uc128"
sheet_name = "Sheet1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(url)
df['announce_date'] = pd.to_datetime(df['announce_date'])
df['first_release_date'] = pd.to_datetime(df['first_release_date'])
df.head()

,name,revision,version,release_series_tag,category,full_name,nick,page_name,announce_date,first_release_date,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,iPad,1st Generation,NaN,iPad,iPad,iPad (1st Generation),iPad,iPad,2010-01-27,2010-04-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,iPad,2nd Generation,2,iPad,iPad,iPad (2nd Generation),iPad 2,iPad 2,2011-03-02,2011-03-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,iPad,3rd Generation,3,iPad,iPad,iPad (3rd Generation),iPad 3,iPad 3,2012-03-07,2012-03-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,iPad,4th Generation,4,iPad,iPad,iPad (4th Generation),iPad 4,iPad 4,2012-10-23,2012-11-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,iPad,5th Generation,5,iPad,iPad,iPad (5th Generation),iPad 5,iPad 5,2017-03-21,2017-03-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [454]:
def get_suggested_links(host, row):
    suggested_links = """
    
You might also like:

"""
    if row.name in ['Apple Watch', 'iPhone']:
         suggested_links += f"- [{row.category} Release Dates]({host}/categories/{row.category.lower().replace(' ','-')}/)\n"
    elif row.name in ['iPad']:
        suggested_links += f"- [{row.name} Release Dates]({host}/tags/{row.name.lower().replace(' ','-')}/)\n"
    else:
        suggested_links += f"- [All {row.category} Release Dates]({host}/categories/{row.category.lower().replace(' ','-')}/)\n"
        suggested_links += f"- [{row.name} Release Dates]({host}/tags/{row.name.lower().replace(' ','-')}/)\n"
    
    suggested_links += f"""- [Releases in {str(row.first_release_date.year)}]({host}/tags/{str(row.first_release_date.year).lower().replace(' ','-')}/)
- [All Apple Products]({host}/categories/)\n"""
    return suggested_links
    

In [475]:
def get_page_content(row):
#     print(type(row.first_release_date))
    host = 'https://AppleReleaseDate.com'
    categories = [row.category]
    tags = [row.name, str(row.first_release_date.year)]
    if row.release_series_tag not in tags:
        tags.append(row.release_series_tag)
    post_content = f"""+++
ShowToc = false
categories = {categories}
date = {str(row.first_release_date)}
title = "{get_page_name(row)}"
tags = {tags}

+++

### The {get_content_name(row)} was released on: 
#### {row.first_release_date.strftime('%B %d, %Y')}


<!--more-->

{get_suggested_links(host=host,row=row)}


<kbd> {row.page_content}</kbd>

"""
    return post_content

In [476]:
def get_content_name(row):
    if row.page_name == row.full_name:
        return row.full_name
    
    return f'{row.page_name} / {row.full_name}'

In [477]:
def get_page_name(row):
    year = str(row.first_release_date.year)
    if year in row.page_name:
        return f'{row.page_name}'
    else:
        return f'{row.page_name} ({year})'

In [480]:
def update_posts(git_push=True):
    export_root = Path('/Users/arjun921/working_directory/apple-release-dates/content/posts')
    commit_message = f"Autogenerated commit message @ {str(datetime.datetime.now())}\n\n"
    for row in df.itertuples():
        print('.',end=' ')
        folder_path = export_root / Path(row.category) 
        post_path = folder_path / Path(get_page_name(row)+'.md')
        os.makedirs(folder_path, exist_ok=True)
        log = f'Generated/Updated post for: {get_page_name(row)}'
        commit_message += f'{log}\n'
        fopen = open(post_path,'w')
        page_content = get_page_content(row)
        fopen.write(page_content)
        fopen.close()
    print('')
    if git_push:
        os.system(f'cd {folder_path.parent.parent.parent} && git add . && git commit -m "{commit_message}" && git push')

In [482]:
update_posts(True)

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
[master 37b7317] Autogenerated commit message @ 2022-01-23 17:21:36.122357
 97 files changed, 1054 insertions(+), 464 deletions(-)
 rename content/posts/Apple Watch/{Apple Watch 2015.md => Apple Watch (2015).md} (61%)
 rename content/posts/Apple Watch/{Apple Watch Series 2 2016.md => Apple Watch Series 2 (2016).md} (62%)
 rename content/posts/Apple Watch/{Apple Watch Series 3 2016.md => Apple Watch Series 3 (2016).md} (62%)
 rename content/posts/Apple Watch/{Apple Watch Series 4 2018.md => Apple Watch Series 4 (2018).md} (62%)
 rename content/posts/Apple Watch/{Apple Watch Series 5 2019.md => Apple Watch Series 5 (2019).md} (62%)
 rename content/posts/Apple Watch/{Apple Watch Series 6 2020.md => Apple Watch Series 6 (2020).md} (62%)
 rename content/posts/Apple Watch/{Apple Watch Series 7 2021.md => App

To https://github.com/arjun921/apple-release-dates.git
   5f181ec..37b7317  master -> master
